## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Honeyswap

12/09/2022

In [1]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [2]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/honeyswap-gnosis'

In [3]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['_meta']['block']['timestamp'], base = 0)))
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-12 16:17:00


True

6-Month Average Daily Volume

In [4]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [5]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [6]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [7]:
df['dailyVolumeUSD'].astype(float).mean()

183189.8698075358

6-Month Average TVL

In [8]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/honeyswap.csv")

In [9]:
lma

,Unnamed: 0,Date,Timestamp,Honeyswap,Honeyswap.1,Honeyswap.2,Honeyswap.3,Honeyswap.4,Honeyswap.5,Honeyswap.6,...,Honeyswap.679,Honeyswap.680,Honeyswap.681,Honeyswap.682,Honeyswap.683,Honeyswap.684,Honeyswap.685,Honeyswap.686,Honeyswap.687,Honeyswap.688
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,Total,xdai,polygon,Total,Total,Total,Total,...,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon,polygon
2,NaN,NaN,NaN,TVL,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,NaN,USDT,MATIC,ROUTE,KRILL,...,HEX,DEFI5,WAULTX,LUNA,MIFARM,ROUTE (POS),WETH,DAI,USDC,SOL
4,NaN,08/04/2021,1.617840e+09,10131675.731226,10131675.731226,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,NaN,08/09/2022,1.662595e+09,6629770.561468702,6515778.170963727,113992.3905049746,2418.626991,113.29992979631375,NaN,NaN,...,44.81084814,NaN,NaN,NaN,0.005154083944181409,NaN,25.130078516243955,1668.9854698379845,62949.722904,NaN
521,NaN,09/09/2022,1.662682e+09,6705977.1010464905,6591386.546642129,114590.55440436157,2422.329654745,114.41720996629557,NaN,NaN,...,44.81084814,NaN,NaN,NaN,0.005154083944181409,NaN,25.206547996063115,1671.1900482907554,62970.093618,NaN
522,NaN,10/09/2022,1.662768e+09,6903963.320695563,6787356.422845993,116606.89784956964,2445.733254965,117.41359217002933,NaN,NaN,...,44.81084814,NaN,NaN,NaN,0.005154083944181409,NaN,24.99520332699297,1675.4062101413908,63071.006771,NaN
523,NaN,11/09/2022,1.662854e+09,7041296.411482292,6923966.370138095,117330.04134419719,2450.0762255969994,118.9910039843776,NaN,NaN,...,44.81084814,NaN,NaN,NaN,0.005154083944181409,NaN,24.840325954329455,1680.3689807201285,62409.184743,NaN


In [10]:
lma.iloc[:,3]

0                   Dexes
1                   Total
2                     TVL
3                     NaN
4         10131675.731226
              ...        
520     6629770.561468702
521    6705977.1010464905
522     6903963.320695563
523     7041296.411482292
524     7023019.366100182
Name: Honeyswap, Length: 525, dtype: object

In [11]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

8536124.66999322

Largest Pool Concentration

In [13]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    if not request.ok:
      continue
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_64532/4280252882.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_64532/4280252882.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_64532/4280252882.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)
/tmp/ipykernel_64532/4280252882.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [14]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x1384d8a5dbd206aa312c700358f3e41ab61756c4,Honeyswap Moons on xDai/Wrapped XDAI,5733.949597332996265642,7505389.60724378121406691,1599154210,11814253
1,0x4505b262dc053998c10685dc5f9098af8ae5c8ad,Honeyswap Honey/Wrapped XDAI,149793.252354595393071532,128009750.9576508329492315330827947,1599171490,11817677
2,0x01f4a4d82a4c1cf12eb2dadc35fd87a14526cc79,Honeyswap USD//C on xDai/Wrapped XDAI,405890.968074354257536958,9771638.4118045061224505355,1599418960,11866830
3,0x7bea4af5d425f2d4485bdad1859c88617df31a67,Honeyswap Wrapped Ether on xDai/Wrapped XDAI,22605.0361606887298574182615431687,29792806.994032099361738088,1599419155,11866869
4,0x5e0b59ad1225c18a5a816e3d28b3212c6a3c9320,Honeyswap dOrgX token/Wrapped XDAI,1.025,0,1599572040,11897248
...,...,...,...,...,...,...
189,0x398a34b3d0f4e5b8df6e914331fceafd8ce8852e,Honeyswap Parahyangan/Wrapped XDAI,0.019,0,1662795560,24121665
190,0x8295a59b07dd7367735aa92d1e0a8b3c2f5b3af8,Honeyswap Adept Coin Governance on xDai/Tharan...,0,0,1662808590,24123945
191,0xca74ad29bd34400e3745f8809764cb3fc55b2a37,Honeyswap Wrapped Ether on xDai/Gnosis Token o...,0.0003568026759303918594243122254727147,0,1662867715,24133613
192,0xce175daea17296fc6769ec0fa1e33950c10dd140,Honeyswap Paxos Gold/DefiPulse Index on xDai,0,0,1662901130,24139042


In [15]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.3598043216556756

In [16]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.8238024230592623

Liquidity Pools with > $1mn TVL

In [17]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

1

Monthly Active Users

In [18]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [19]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [20]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [21]:
df['dailyActiveUsers'].astype(float).mean()

346.3879781420765

Implied slippage

In [22]:
query = """
{
  liquidityPools(first:10, orderBy: totalValueLockedUSD, orderDirection: desc){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
    inputTokens{
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    inputTokenBalances
  }
}
"""
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()

In [23]:
data

{'data': {'liquidityPools': [{'id': '0x28dbd35fd79f48bfa9444d330d14683e7101d817',
    'name': 'Honeyswap Wrapped Ether on xDai/Gnosis Token on xDai',
    'totalValueLockedUSD': '2168110.131589464904852097773354033',
    'cumulativeVolumeUSD': '9698989.058406998785890802738090433',
    'createdTimestamp': '1614798970',
    'createdBlockNumber': '14830802',
    'inputTokens': [{'id': '0x6a023ccd1ff6f2045c3309768ead9e68f978f6e1',
      'name': 'Wrapped Ether on xDai',
      'symbol': 'WETH',
      'lastPriceUSD': '1736.656504891638647815063453921369',
      'decimals': 18},
     {'id': '0x9c58bacc331c9aa871afd802db6379a98e80cedb',
      'name': 'Gnosis Token on xDai',
      'symbol': 'GNO',
      'lastPriceUSD': '169.5812391917517980273860838625467',
      'decimals': 18}],
    'inputTokenBalances': ['624219621290264153100', '6392541244311531270656']},
   {'id': '0x5aa67e24ba8a3fbdc553e308d02377e03ce9e94f',
    'name': 'Honeyswap Giveth from Mainnet/Gnosis Token on xDai',
    'totalValueL

In [25]:
slip1000 = []
slip10000 = []
slip100000 = []

for pool in data['data']['liquidityPools']:
    token0 = pool['inputTokens'][0]
    token0_reserveUSD = (float(pool['inputTokenBalances'][0])/10**token0['decimals'])*float(token0['lastPriceUSD'])
    
    token1 = pool['inputTokens'][1]
    token1_reserveUSD = (float(pool['inputTokenBalances'][1])/10**token1['decimals'])*float(token1['lastPriceUSD'])

    try:
        pi1000 = 1000/token0_reserveUSD
    except:
        pi1000 = 1000/token1_reserveUSD
    
    try:
        pi10000 = 10000/token0_reserveUSD
    except:
        pi10000 = 10000/token1_reserveUSD
    
    try:
        pi100000 = 100000/token0_reserveUSD
    except:
        pi100000 = 100000/token1_reserveUSD
    
    slip1000.append(pi1000)
    slip10000.append(pi10000)
    slip100000.append(pi100000)


In [26]:
np.mean(slip1000)

0.007380850137334008

In [27]:
np.mean(slip10000)

0.0738085013733401

In [28]:
np.mean(slip100000)

0.738085013733401